In [1]:
import requests
from dotenv import load_dotenv
import os
import json
import pandas as pd
import sqlite3
from datetime import datetime

In [3]:
# this loads in environment variables
# fancy way to keep tokens secret and secure
load_dotenv()

True

# Authentication with Concur API
This relies on the refresh token being refreshed at least 1 every 6 months. If this expires a new JWT Company Token would need to be produced by going onto Concur as an authentiaction admin.

Use the refresh token to get a new access token. Access tokens expire every hour and can be refreshed using the refresh token.

This resets the timer on both tokens as it writes a new access and refresh token back to the token.json file.

In [4]:
# get vars from environment variables and reads in token values from JWT
token_url = r"https://us2.api.concursolutions.com/oauth2/v0/token"
client_id = os.getenv('client_id')
client_secret = os.getenv('client_secret')
with open("token.json", "r") as f:
    read_token = json.load(f)
    refresh_token = read_token['refresh_token']

token_headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Connection": "close"
}

token_data = {
    "client_id": client_id,
    "client_secret": client_secret,
    "grant_type": "refresh_token",
    "refresh_token": refresh_token,
}

# make request using refresh token to get new access token
# access token needs to be refreshed every hour
token_response = requests.post(token_url, headers=token_headers, data=token_data)

token_json = token_response.json()
token_write = json.dumps(token_json)

access_token = token_json['access_token']
refresh_token = token_json['refresh_token']

with open("token.json", "w") as f:
    f.write(token_write)
    
heads = {
    "Accept": "application/json",
    "Authorization": "Bearer " + access_token}

# Get data from Concur API
This first gets a list of all unsubmitted reports and then makes a request for each via a for loop to get the "isPendingDelegatorReview" status.

In [5]:
# get all reports with unsubmitted status
reports_url = r"https://www.concursolutions.com/api/v3.0/expense/reports?limit=100&user=ALL&approvalStatusCode=A_NOTF"
reports_r = requests.get(reports_url, headers=heads)
reports_json = reports_r.json()
rep_data = []
rep_data += reports_json["Items"]

while reports_json["NextPage"] != None:
    reports_url = reports_json["NextPage"]
    reports_r = requests.get(reports_url, headers=heads)
    reports_json = reports_r.json()
    
    rep_data += reports_json["Items"]
    

# get updates on all of the reports that are waiting to be submitted
reports_v4_url = r"https://us2.api.concursolutions.com/expensereports/v4/reports/"
rep_del_list = []
for rep in rep_data:
    
    # build request
    rep_id = rep["ID"]
    url = reports_v4_url + rep_id
    rep_json = requests.get(url, headers=heads).json()
    
    # append report ID and status to list as json object
    txn = {}
    status = rep_json["isPendingDelegatorReview"]
    txn['report_id'] = rep_id
    txn['is_pending_delegator_review'] = status
    
    txn['date_extracted_at'] = datetime.now()
    rep_del_list.append(txn)

# Write to DB

In [6]:
# write data to dataframe
df = pd.DataFrame.from_dict(rep_del_list)

# create connection to sqlite db and overwrite table
conn = sqlite3.connect("B:\Finance\Firm Integration\Expense & GL Project\99. db\system_config.sqlite")
df.to_sql("report_delegator_status", conn, if_exists="replace", index=False)

conn.close()

In [7]:
df.head()

,report_id,is_pending_delegator_review,date_extracted_at
0,93204E27263B4B2EBCBF,False,2023-07-12 10:50:24.402976
1,3D6F8A598E7E49528903,False,2023-07-12 10:50:24.663359
2,03D9F9835D2A4DA8A587,False,2023-07-12 10:50:24.989321
3,16F5AFA4C3BE4CB0B426,False,2023-07-12 10:50:25.262395
4,64C082C0C38B497CA623,False,2023-07-12 10:50:25.547779
